In [ ]:
# in this notebook I will clean the original parks dataset

In [2]:
import numpy as np
import pandas as pd

In [3]:
# loading raw data and parsing dates:

raw_data = pd.read_csv("../data/Social_Distancing__Parks_Crowds_Data_original.csv",parse_dates = ['encounter_timestamp'])

In [4]:
# inspecting data and making sure parse dates worked properly:

raw_data.head(10)

,encounter_timestamp,park_area_id,park_district,park_borough,patroncount,in_playground,action_taken,amenity
0,2020-07-07 14:33:23,X179,X-02,Bronx,50,True,Approached the crowd; they ignored the employee,Basketball court
1,2020-07-05 10:40:24,B247,B-18,Brooklyn,50,False,Approached the crowd; they ignored the employee,Baseball field
2,2020-07-04 12:56:31,B247,B-18,Brooklyn,20,False,Approached the crowd; they ignored the employee,Bench/sitting area
3,2020-07-03 22:16:52,Q377,Q-06,Queens,75,False,Did not approach the crowd; the crowd remains,Soccer field
4,2020-07-02 15:11:30,B018,B-18,Brooklyn,75,False,Approached the crowd; they ignored the employee,Open field/multi-purpose play area
5,2020-07-01 11:35:04,X039-ZN01,X-15,Bronx,200,False,Did not approach the crowd; the crowd remains,Track/field area
6,2020-06-30 08:15:21,M029-01,M-12,Manhattan,12,True,Did not approach the crowd; the crowd remains,Adult fitness equipment
7,2020-06-29 11:15:32,Q357B01,Q-06,Queens,1,False,Approached the crowd; they complied with instr...,Handall court
8,2020-06-29 07:35:49,M144-ZN04,M-03,Manhattan,15,False,Approached the crowd; they ignored the employee,Baseball field
9,2020-06-28 20:43:04,X092-09,X-13,Bronx,50,True,Did not approach the crowd; the crowd remains,Playground


In [5]:
# deleting any rows that contain a gathering of less than 3 ppl or over 50 ppl
# (an employee cannot count and/or approach a gathering of more than 50)

In [6]:
# Checking data types for each column:

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1869 entries, 0 to 1868
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   encounter_timestamp  1869 non-null   datetime64[ns]
 1   park_area_id         1869 non-null   object        
 2   park_district        1869 non-null   object        
 3   park_borough         1869 non-null   object        
 4   patroncount          1869 non-null   int64         
 5   in_playground        1869 non-null   bool          
 6   action_taken         1869 non-null   object        
 7   amenity              1182 non-null   object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(5)
memory usage: 104.2+ KB


In [7]:
# Checking the number of observations that need to be dropped:

(raw_data['patroncount'] < 3).sum(), (raw_data['patroncount'] > 51).sum()

(119, 244)

In [8]:
# Checking the proportion of observations that need to be dropped:

(raw_data['patroncount'] < 3).sum()/len(raw_data) * 100, (raw_data['patroncount'] > 51).sum()/len(raw_data) * 100

(6.367041198501873, 13.055109684323169)

In [9]:
# 6% were gathering of less than 3 ppl
# 13% were gathering of more than 50 ppl

In [10]:
# Checking what are these gatherings of over 50 ppl:

print(raw_data['patroncount'][raw_data['patroncount'] > 51].describe()),
print(raw_data[raw_data['patroncount'] > 51]['amenity'].value_counts())

count     244.00000
mean      165.52459
std       399.46138
min        53.00000
25%        75.00000
50%       100.00000
75%       200.00000
max      6000.00000
Name: patroncount, dtype: float64
Open field/multi-purpose play area    43
Walking path                          33
Soccer field                          30
Track/field area                      28
Baseball field                        27
Playground                            13
Bench/sitting area                    11
Adult fitness equipment                9
Basketball court                       6
Handall court                          1
Name: amenity, dtype: int64


In [11]:
# dropping too samll and too big gatherings:

data = raw_data[(raw_data['patroncount'] < 51) & 
                (raw_data['patroncount'] > 2) ] 

In [12]:
# Verifying that the new dataset length makes sense:

len(data), len(raw_data)

(1506, 1869)

In [13]:
# checking distribution of cases by borough:

data['park_borough'].value_counts()/len(data) * 100

Queens           35.856574
Manhattan        24.435591
Brooklyn         14.741036
Bronx            14.010624
Staten Island    10.956175
Name: park_borough, dtype: float64

In [14]:
# comparing the % of cases by borough to the % of population in each borough:
# population data is from "open nyc": https://data.cityofnewyork.us/City-Government/2020-population/t8c6-3i7b

pop = pd.read_csv("2020_population_original.csv")

In [15]:
pop = pop.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

pop_queens = pop[(pop['Borough'] == 'Queens') &
             (pop['Age Group'] == 'Total')]['2020'].values[0]
pop_manhattan = pop[(pop['Borough'] == 'Manhattan') &
                    (pop['Age Group'] == 'Total')]['2020'].values[0]
pop_brooklyn = pop[(pop['Borough'] == 'Brooklyn') &
                   (pop['Age Group'] == 'Total')]['2020'].values[0]
pop_bronx = pop[(pop['Borough'] == 'Bronx') &
                (pop['Age Group'] == 'Total')]['2020'].values[0]
pop_si = pop[(pop['Borough'] == 'Staten Island') &
             (pop['Age Group'] == 'Total')]['2020'].values[0]


pop_by_borough = [pop_queens,pop_manhattan,pop_brooklyn,pop_bronx,pop_si,]
total_pop = np.array(pop_by_borough).sum()
pop_by_borough, total_pop


([2330295, 1638281, 2648452, 1446788, 487155], 8550971)

In [16]:
# comparing % of population by borough to % of cases by borough:

pop_by_borough/total_pop * 100 - data['park_borough'].value_counts()/len(data) * 100 

# about 10% change, not too bad. 
# Need to mention it in the discussion. Could be attributed to: 
                                                                #1. more policing (more employees)
                                                                #2. change in population prefrences
                                                                #3. more parks available 

Queens           -8.604756
Manhattan        -5.276586
Brooklyn         16.231493
Bronx             2.908951
Staten Island    -5.259103
Name: park_borough, dtype: float64

In [17]:
# inspecting how many reports on gathering each month:

data['encounter_timestamp'].dt.month.value_counts().sort_index()

3     381
4     565
5     292
6     248
7      15
8       2
9       2
10      1
Name: encounter_timestamp, dtype: int64

In [18]:
# we can only use march, april, may and june.
# need to mention in discussion that collection stopped in July.

In [19]:
# checking numbers that would be dropped:

len(data[data['encounter_timestamp'].dt.month < 7]), len(data[data['encounter_timestamp'].dt.month > 6])

(1486, 20)

In [20]:
# dropping:

data = data[data['encounter_timestamp'].dt.month < 7]

In [21]:
# Making sure dropping went poperly:

len(data)

1486

In [22]:
# setting time as index and sorting:

data = data.sort_values('encounter_timestamp')

In [23]:
#inspecting data:

data

,encounter_timestamp,park_area_id,park_district,park_borough,patroncount,in_playground,action_taken,amenity
1842,2020-03-25 08:07:40,M108T01,M-15,Manhattan,17,True,Approached the crowd; they complied with instr...,NaN
1841,2020-03-25 10:44:20,R030-099,R-02,Staten Island,18,False,Approached the crowd; they ignored the employee,NaN
1840,2020-03-25 10:49:03,R045-01,R-02,Staten Island,8,True,Approached the crowd; they ignored the employee,NaN
1839,2020-03-25 10:49:53,R005,R-01,Staten Island,6,False,Did not approach the crowd; the crowd remains,NaN
1838,2020-03-25 10:54:28,R005-ZN01,R-01,Staten Island,10,False,Did not approach the crowd; the crowd remains,NaN
...,...,...,...,...,...,...,...,...
18,2020-06-28 12:17:33,M144-ZN04,M-03,Manhattan,25,False,Approached the crowd; they ignored the employee,Baseball field
17,2020-06-28 12:18:20,M144-ZN06,M-03,Manhattan,20,False,Approached the crowd; they ignored the employee,Open field/multi-purpose play area
9,2020-06-28 20:43:04,X092-09,X-13,Bronx,50,True,Did not approach the crowd; the crowd remains,Playground
8,2020-06-29 07:35:49,M144-ZN04,M-03,Manhattan,15,False,Approached the crowd; they ignored the employee,Baseball field


In [24]:
# dropping subarea of park:

data['park_area_id'] = data['park_area_id'].apply(lambda x: x.split("-")[0])

In [25]:
len(data['park_area_id'].unique()), len(data['park_area_id'])

(373, 1486)

In [26]:
# merging data with location of parks csv:

locations_directory = pd.read_csv('../output/imap.csv')

In [27]:
data = data.merge(locations_directory, on = 'park_area_id')

In [28]:
data = data.rename(columns = {'encounter_timestamp':'timestamp',
                              'facility__Name':'name',
                              'facility__Location':'location',
                              'facility__Zip':'zip',
                              'ADDRESS_Ext':'address'})

In [29]:
data.drop(columns = 'Unnamed: 0', inplace = True)

In [30]:
len(data)

1486

In [31]:
data.to_csv('../output/clean_parks_data.csv')